In [1]:
import tensorflow as tf
sess = tf.InteractiveSession()

In [9]:
import json
from pprint import pprint

with open('touchdata.json') as data_file:    
    data = json.load(data_file)
pprint(data['submission'])
#drawing = data['submission'][0][0]['strokes']
#pprint(drawing)

[[{'client': 'Qt Recorder',
   'description': '0',
   'strokes': [[{'p': 1, 't': 2112, 'x': 184, 'y': 179.00000000000003},
                {'p': 1,
                 't': 2127,
                 'x': 183.33333333333334,
                 'y': 176.33333333333334},
                {'p': 1,
                 't': 2159,
                 'x': 182.33333333333334,
                 'y': 172.66666666666666},
                {'p': 1,
                 't': 2171,
                 'x': 178.33333333333334,
                 'y': 167.33333333333331},
                {'p': 1,
                 't': 2205,
                 'x': 154.66666666666666,
                 'y': 166.33333333333334},
                {'p': 1,
                 't': 2223,
                 'x': 134.66666666666669,
                 'y': 180.66666666666666},
                {'p': 1,
                 't': 2244,
                 'x': 111.33333333333333,
                 'y': 209.33333333333334},
                {'p': 1, 't': 2274, 'x': 93, 'y':

In [3]:
from copy import copy

mini = copy(drawing[0][0])
maxi = copy(drawing[0][0])
for s in drawing:
    for p in s:
        mini['x'] = min(mini['x'], p['x'])
        mini['y'] = min(mini['y'], p['y'])
        mini['t'] = min(mini['t'], p['t'])
        maxi['x'] = max(maxi['x'], p['x'])
        maxi['y'] = max(maxi['y'], p['y'])
        maxi['t'] = max(maxi['t'], p['t'])
pprint(mini)
pprint(maxi)

{'p': 1, 't': 2112, 'x': 93, 'y': 140.66666666666666}
{'p': 1, 't': 2466, 'x': 226.33333333333331, 'y': 360.3333333333333}


In [4]:
for s in drawing:
    for p in s:
        p['x'] = 10 * (p['x'] - mini['x']) / (maxi['x'] - mini['x'])
        p['y'] = 10 * (p['y'] - mini['y']) / (maxi['y'] - mini['y'])
        p['t'] = 10 * (p['t'] - mini['t']) / (maxi['t'] - mini['t'])

pprint(drawing)

[[{'p': 1, 't': 0.0, 'x': 6.825000000000001, 'y': 1.7450682852807302},
  {'p': 1,
   't': 0.423728813559322,
   'x': 6.775000000000002,
   'y': 1.6236722306525047},
  {'p': 1,
   't': 1.3276836158192091,
   'x': 6.700000000000002,
   'y': 1.4567526555386952},
  {'p': 1,
   't': 1.6666666666666667,
   'x': 6.400000000000002,
   'y': 1.2139605462822454},
  {'p': 1,
   't': 2.6271186440677967,
   'x': 4.624999999999999,
   'y': 1.1684370257966625},
  {'p': 1,
   't': 3.135593220338983,
   'x': 3.1250000000000018,
   'y': 1.8209408194233687},
  {'p': 1,
   't': 3.7288135593220337,
   'x': 1.3749999999999998,
   'y': 3.125948406676784},
  {'p': 1, 't': 4.576271186440678, 'x': 0.0, 'y': 5.842185128983309},
  {'p': 1, 't': 5.254237288135593, 'x': 1.75, 'y': 8.573596358118362},
  {'p': 1,
   't': 6.15819209039548,
   'x': 3.1249999999999996,
   'y': 9.377845220030347},
  {'p': 1, 't': 6.468926553672317, 'x': 4.800000000000001, 'y': 10.0},
  {'p': 1,
   't': 6.9209039548022595,
   'x': 6.475000

In [8]:
import numpy as np
import operator
from functools import reduce

def line(s, e, cb):
    diff = [abs(i-j) for (i,j) in zip(s, e)]
    sign = [1 if i < j else -1 for (i,j) in zip(s, e)]
    maxd = max(diff)
    p = [maxd/2]*len(s)
    q = list(s)
    
    for _ in range(maxd):
        cb(tuple(q))
        for i in range(len(s)):
            p[i] -= diff[i]
            if (p[i] < 0):
                p[i] += maxd
                q[i] += sign[i]

# anti aliased line
def aline(s, e, cb):
    def _fpart(x):
        return x - int(x)
    def _rfpart(x):
        return 1 - _fpart(x)
    
    diff = [abs(j-i) for (i,j) in zip(s, e)]
    maxd = max(diff)
    if maxd <= 0: return
    maxi = np.argmax(diff)
    sign = 1 if s[maxi] < e[maxi] else -1
    grad = [(j-i) / maxd for (i,j) in zip(s, e)]
    q = [s[i] + _rfpart(s[maxi]) * grad[i] for i in range(len(s))]
    q[maxi] -= sign
    
    pprint("----")
    pprint(s)
    pprint(e)
    pprint(grad)
    pprint(maxi)
    pprint(q)
    
    for _ in range(round(s[maxi]) + sign, round(e[maxi]), sign):
        for n in range(2**len(s)):
            i = reduce(operator.mul, [_fpart(v) if (n & (2**a)) else _rfpart(v) for (v, a) in zip(q, range(len(q)))])
            if i == 0: continue
            p = [int(q[a] + (1 if (n & (2**a)) else 0)) for a in range(len(q))]
            pprint(p)
            pprint(i)
            cb(tuple(p), i)
        q = [v + g for (v,g) in zip(q,grad)]
                
picture = np.zeros((11, 11, 11))
def pixel(p, a = 1):
    #pprint(p)
    picture[p] += a
    
for stroke in drawing:
    for (s, e) in zip(stroke, stroke[1:]):
        #spoint = (round(s['t']), round(s['y']), round(s['x']))
        #epoint = (round(e['t']), round(e['y']), round(e['x']))
        #line(spoint, epoint, pixel)
        spoint = (s['t'], s['y'], s['x'])
        epoint = (e['t'], e['y'], e['x'])
        aline(spoint, epoint, pixel)

np.set_printoptions(threshold=np.nan, linewidth=140, precision=4, suppress=True)
print(picture)

'----'
(0.0, 1.7450682852807302, 6.825000000000001)
(0.423728813559322, 1.6236722306525047, 6.775000000000002)
[1.0, -0.2864946889226123, -0.1179999999999975]
0
[0.0, 1.458573596358118, 6.707000000000003]
'----'
(0.423728813559322, 1.6236722306525047, 6.775000000000002)
(1.3276836158192091, 1.4567526555386952, 6.700000000000002)
[1.0, -0.18465477996965174, -0.08296875000000019]
0
[0.0, 1.517261001517451, 6.727187500000002]
'----'
(1.3276836158192091, 1.4567526555386952, 6.700000000000002)
(1.6666666666666667, 1.2139605462822454, 6.400000000000002)
[1.0, -0.7162367223065269, -0.8849999999999996]
0
[1.0, 0.9752149721800698, 6.105000000000002]
'----'
(1.6666666666666667, 1.2139605462822454, 6.400000000000002)
(2.6271186440677967, 1.1684370257966625, 4.624999999999999)
[0.5410997055781005, -0.02564705379469451, -1.0]
2
[1.991326490013526, 1.1985723140054287, 6.800000000000004]
'----'
(2.6271186440677967, 1.1684370257966625, 4.624999999999999)
(3.135593220338983, 1.8209408194233687, 3.12500

In [6]:
list1 = [1,2,3,4,5]
pprint(list(zip(list1, list1[1:])))


[(1, 2), (2, 3), (3, 4), (4, 5)]


In [7]:
a = np.zeros((3,3))
a[tuple([1,1])] = 1
a
list((2,2))
np.argmax([1,4,2])

import operator
from functools import reduce

reduce(operator.mul, [1,2,3])

6